# Example Norway"

In [1]:
import dataverk as dv
from dataverk_tools.themes import nav

import pandas as pd
import ipywidgets as widgets

import json
import plotly.graph_objs as go
import plotly.io as plio
import plotly_express as px
import plotly.figure_factory as ff

from datetime import datetime
from keplergl import KeplerGl 

# ETL

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/datasett/maps/master/norway/data/kommune_dummy_data.csv')

In [3]:
df.head()

,kommunenummer,value,color
0,101,40,"[247, 103, 134]"
1,104,4,"[199, 18, 62]"
2,105,89,"[156, 169, 126]"
3,106,24,"[157, 157, 184]"
4,111,5,"[114, 29, 136]"


In [4]:
data = df.to_json(orient='records')

In [5]:
url = 'https://raw.githubusercontent.com/datasett/maps/master/norway/data/counties_2018_s_geojson.json'
ds = pd.read_json(url)

In [6]:
def getData(row):
    return row['properties']['centroid_kommune']

In [7]:
ds['centroid_kommune'] = ds['features'].apply(getData)

In [8]:
#ds.head(1)['features'][0]

In [9]:
dt = ds.copy()

In [10]:
def getId(row):
    return row['features']['properties']['kommunenummer']
    
def getGeom(row):
    return json.dumps(row['features']['geometry'])

In [11]:
dt['id'] = dt.apply(getId, axis=1)
dt['kommune'] = dt.apply(getId, axis=1)
dt['geometry_string'] = dt.apply(getGeom, axis=1)
dt.head(1)

,type,features,centroid_kommune,id,kommune,geometry_string
0,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poly...","[11.498404769493327, 59.08595732263681]",0101,0101,"{""type"": ""Polygon"", ""coordinates"": [[[11.55794..."


In [12]:
df_with_geometry = dt[['id','kommune','geometry_string']]

# add to map
from keplergl import KeplerGl
map_1 = KeplerGl()
map_1.add_data(df_with_geometry, "kommuner")
map_1

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'kommuner': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20…

# Create datapackage

In [28]:
readme = """
# Eksempler på kartvisninger med webGL
"""

metadata = {
'store': 'gs',
'repo': 'https://raw.githubusercontent.com/datasett/nav-examples/master/',
'readme': readme,
'license':'MIT',
'accessRights':'Open',
'author': 'paul.bencze@nav.no',
'title':'Eksempler på kart med webGL',
'title':'Eksempler på kart med webGL',
'description':'Eksempler på kartvisninger med webGL',
'name':'Eksempler på kartvisninger',
'source':'NAV',
'keywords':['kart, eksempler', 'examples'],
'provenance':'NAV',
'publisher': 'NAV',
'bucket': 'nav-opendata',
}

In [29]:
dp = dv.Datapackage(metadata)

In [30]:
spec = {
    "data": data,
    "map": "https://raw.githubusercontent.com/datasett/maps/master/norway/data/counties_2018_s_geojson.json",
    "colorby": "kommunenummer",
    "value": 'kommunenummer',
    "lineColor": [200,200,200], 
    "initialViewState": {
      "longitude": 14,
      "latitude": 65,
      "zoom": 3.5,
      "pitch": 0,
      "bearing": 0,
    }
  }

In [31]:
description = '''
Kart beskrivelse
'''

dp.add_view(
    spec_type='map',
    name='Kommuner farge etter kommunenummer',
    description = description,
    resources=[],
    attribution = "Kilde: NAV.",
    spec=spec
)

In [32]:
#GOOGLE_APPLICATION_CREDENTIALS environment variable must be defined before initializing the client

In [33]:
client = dv.Client()
client.publish(dp)

gs
2019-08-09T09:22:40.528415: B149899 <class 'dataverk.connectors.google_storage.GoogleStorageConnector'>: String (format: json) written to https://storage.googleapis.com/nav-opendata/26de02c184aceed17b9f968161f623b2/datapackage.json
2019-08-09T09:22:41.463444: B149899 <class 'dataverk.connectors.elasticsearch.ElasticsearchConnector'>: Document 26de02c184aceed17b9f968161f623b2 of type odata indexed to elastic index: odata.
